**Load Libs**

In [29]:
import os
import requests
import pandas as pd
import time
import json

**Paths**

In [30]:
DATA = '..\\..\\data'
TOKEN = f'{DATA}\\token'
TO_SCRAP = f'{DATA}\\to_scrap'
SCRAPED = f'{DATA}\\scraped'
ERRORS = f'{DATA}\\errors'

In [31]:
print(os.path.abspath(TOKEN))

C:\Users\juanm\OneDrive\Bureau\ESGI - Projets\4IABD\Projet Annuel\data\token


**Generate Token For Spotify API Requests**

In [32]:
# Generate token
def generate_token() -> str:
    f = open(f'{TOKEN}\\auth_spotify_api.json')
    auth_spotify_api = json.load(f)
    return requests.post(
        'https://accounts.spotify.com/api/token',
        data={
            'Content_type': 'application/x-www-form-urlencoded',
            'grant_type': 'client_credentials',
            'client_id': auth_spotify_api.get('client_id'),
            'client_secret': auth_spotify_api.get('client_secret'),
        },
    ).json()['access_token']

In [33]:
def regen_token(token: str, timestamp: float) -> tuple[str, float]:
    t = time.time()
    return (generate_token(), t) if t - timestamp >= 3600 else (token, timestamp)

In [34]:
spotify_api = 'https://api.spotify.com/v1'

**Load Dataset To Scrap**

In [35]:
df = pd.read_csv(f'{TO_SCRAP}\\custom_music_dataset.csv')

**Retrieve Dates And SongID**

In [36]:
# Create empty csv with columns
pd.DataFrame({
    'artist_name': pd.Series(dtype='str'),
    'track_name': pd.Series(dtype='str'),
    'release_date': pd.Series(dtype='str'),
    'genre': pd.Series(dtype='str'),
    'spotify_id': pd.Series(dtype='str')
}).to_csv(f'{SCRAPED}\\complete_custom_dataset.csv', index=False)

In [43]:
timestamp = time.time()
token = generate_token()
for rows in df.values:
    token, timestamp = regen_token(token, timestamp)
    response = requests.get(f'{spotify_api}/search?q=track:{rows[0]}%20artist:{rows[1]}&type=track',
                            headers={
                                'Authorization': f'Bearer {token}'
                            })
    if response.status_code == 429:
        print('Too Many Request')
        break
    if response.status_code == 200 and response.json()['tracks']['total'] > 0:
        album_dict = response.json()['tracks']['items'][0]['album']
        release_date = f"{str(sorted([int(data['album']['release_date'][:4]) for data in response.json()['tracks']['items']])[0])[:3]}0"
        pd.DataFrame({'artist_name': rows[1],
                      'track_name': rows[0],
                      'release_date': release_date,
                      'genre': album_dict['genres'][0] if 'genres' in album_dict
                      else '',
                      'spotify_id': response.json()['tracks']['items'][0]['id']}, index=[0]) \
            .to_csv(f'{SCRAPED}\\complete_custom_dataset.csv', index=False, header=False, mode='a')
    else:
        pd.DataFrame({'artist_name': rows[1],
                      'track_name': rows[0],
                      'release_date': '',
                      'genre': '',
                      'error_type': response.status_code}, index=[0]) \
            .to_csv(f'{ERRORS}\\error_scrapping_custom.csv', index=False, header=False, mode='a')

Too Many Request
